Φόρτωση δεδομένων και δημιουργία PySpark Session

In [ ]:
!pip -q install pyspark
from pyspark.sql import SparkSession
from google.colab import files

In [ ]:
spark = SparkSession.builder.master("local[*]").appName("Spark SQL").getOrCreate()
print("Spark session started")

Spark session started


In [ ]:
uploaded = files.upload()
filename = list(uploaded.keys())
df = spark.read.csv(filename, header=True, inferSchema=True)
print("CSV file loaded into DataFrame")

Saving telecom_new_col.csv to telecom_new_col.csv
CSV file loaded into DataFrame


**Βήμα 1ο: Δημιουργία προσωρινού view**

In [ ]:
df.createOrReplaceTempView("churn_view")
spark.catalog.listTables()

spark.sql("SELECT * FROM churn_view").show()

+-----------+------+----+-------+-------------+--------------+------------+----------+------+---------------+-------------+--------------+-------------+--------------+-----+------------+--------------------+--------------+
|CUSTOMER_ID|GENDER| AGE|COUNTRY|TENURE_MONTHS| CONTRACT_TYPE|HAS_INTERNET|HAS_MOBILE|HAS_TV|MONTHLY_CHARGES|TOTAL_CHARGES|NUM_COMPLAINTS|SUPPORT_CALLS|PAYMENT_METHOD|CHURN|NUM_SERVICES|AVG_CHARGE_PER_MONTH|IS_LONG_TENURE|
+-----------+------+----+-------+-------------+--------------+------------+----------+------+---------------+-------------+--------------+-------------+--------------+-----+------------+--------------------+--------------+
|    C000001|  Male|68.0|     FR|         61.0|Month-to-month|           0|         1|     0|          17.54|      1077.63|             0|            0| Bank transfer|  1.0|           1|  17.666065573770492|             1|
|    C000002|Female|65.0|     FR|         37.0|Month-to-month|           1|         1|     0|          50.82

**Βήμα 2ο: Ερωτήματα - Queries**



*   2.1 Churn ανά τύπο συμβολαίου



In [ ]:
query = """ SELECT CONTRACT_TYPE,
            COUNT(*) AS total_customers,
            SUM(CASE WHEN (CHURN=1) THEN 1 ELSE 0 END) AS cust_churn1,
            ROUND(AVG(CHURN)*100, 2) AS churn_rate
            FROM churn_view
            GROUP BY CONTRACT_TYPE
            ORDER BY churn_rate DESC"""

spark.sql(query).show()

+--------------+---------------+-----------+----------+
| CONTRACT_TYPE|total_customers|cust_churn1|churn_rate|
+--------------+---------------+-----------+----------+
|Month-to-month|           5326|       2834|     53.21|
|      One year|           2440|        503|     20.61|
|      Two year|           1934|        264|     13.65|
+--------------+---------------+-----------+----------+





*   2.2 Churn ανά πλήθος υπηρεσιών



In [ ]:
query = """ SELECT NUM_SERVICES,
            ROUND(AVG(CHURN)*100, 2) AS churn_rate
            FROM churn_view
            GROUP BY NUM_SERVICES
            ORDER BY NUM_SERVICES"""

spark.sql(query).show()

+------------+----------+
|NUM_SERVICES|churn_rate|
+------------+----------+
|           0|     43.23|
|           1|     47.98|
|           2|     37.38|
|           3|     25.12|
+------------+----------+



*   2.3 Churn και μηνιαία χρέωση

In [ ]:
query = """ SELECT INT(CHURN),
            ROUND(AVG(MONTHLY_CHARGES), 2) AS avg_monthly_charges
            FROM churn_view
            GROUP BY CHURN
            ORDER BY CHURN"""

opt_query = """ SELECT CONTRACT_TYPE,
                INT(CHURN) AS CHURN,
                ROUND(AVG(MONTHLY_CHARGES), 2) AS avg_monthly_charges
                FROM churn_view
                GROUP BY CONTRACT_TYPE, CHURN
                ORDER BY CONTRACT_TYPE, CHURN"""

spark.sql(query).show()
spark.sql(opt_query).show()

+-----+-------------------+
|CHURN|avg_monthly_charges|
+-----+-------------------+
|    0|              36.26|
|    1|              37.02|
+-----+-------------------+

+--------------+-----+-------------------+
| CONTRACT_TYPE|CHURN|avg_monthly_charges|
+--------------+-----+-------------------+
|Month-to-month|    0|              41.21|
|Month-to-month|    1|              38.48|
|      One year|    0|              35.25|
|      One year|    1|              33.44|
|      Two year|    0|              30.04|
|      Two year|    1|              28.12|
+--------------+-----+-------------------+





*   3 Γεωγραφική Ανάλυση (Top-X περιοχές)


In [ ]:
query = """ SELECT COUNTRY,
            COUNT(*) AS total_customers,
            ROUND(AVG(CHURN)*100, 2) AS churn_rate
            FROM churn_view
            GROUP BY COUNTRY
            HAVING total_customers > 100
            ORDER BY churn_rate DESC
            LIMIT 5"""

spark.sql(query).show()

+-------+---------------+----------+
|COUNTRY|total_customers|churn_rate|
+-------+---------------+----------+
|     IT|           1003|     40.78|
|     DE|           1198|     37.65|
|     GR|           3732|     37.03|
|     UK|           1560|     36.15|
|     ES|           1050|      36.1|
+-------+---------------+----------+



***Σχολιασμός αποτελεσμάτων***



*   Ερώτημα 2.1

Οι πελάτες με Month-to-Month συμβόλαιο φαίνεται πράγματι να έχουν υψηλότερο churn_rate (ρυθμό αποχώρησης) σε σχέση με τους υπόλοιπους πελάτες που έχουν συμβόλαιο τύπου One/Two Year. Συγκεκριμένα, οι Month-to-Month πελάτες έχουν churn_rate = 53.21%, οι One-Year πελάτες έχουν churn_rate = 20.61% και οι Two-Year πελάτες έχουν churn_rate = 13.65%.

*   Ερώτημα 2.2

Πράγματι, οι πελάτες που συνδυάζουν περισσότερες από 3 υπηρεσίες έχουν το χαμηλότερο churn_rate = 25.12%. Ακολουθούν αυτοί με 2 υπηρεσίες με churn_rate = 37.38%. Οι πελάτες που δεν συνδυάζουν καμία υπηρεσία έχουν churn_rate = 43.23%, ενώ αυτοί που έχουν μόνο μία υπηρεσία έχουν churn_rate = 47.98% (το υψηλότερο μέχρι στιγμής).

*   Ερώτημα 2.3

Οι πελάτες με churn που προσεγγίζει το 1 (δηλ. τείνουν να αποχωρήσουν) φαίνεται ότι έχουν υψηλότερες χρεώσεις κατά μέσο όρο (avg_monthly_charges). Με βάση τα αποτελέσματα, 37.02 χρηματικές μονάδες δαπανούνται από τους πελάτες που έχουν churn κοντά στο 1, ενώ οι πελάτες που έχουν churn κοντά στο 0 δαπανούν 36.26 χρηματικές μονάδες κατά μέσο όρο.